# Data preprocessing

## Defining the target variable
1. Analyzing the citation count distribution
2. Group citations into categories
3. Create ordinal target variable

## Feature Extraction
1. Extracting page count and number of figures
2. Extracting the number of authors
3. 

## Data cleaning

1. Remove duplicates
2. Deal with outliers
3. Deal with missing data
4. ...

## Textual Data processing

1. Tokenization
2. Stopword removal
3. Stemming
4. Lemmatization
5. Vectorization

## Feature Selection and dimensionality reduction
Analyze the data!


## Model training and evaluation

1. Split data into train, val and test set
2. Select appropriate model candidates
3. Train models, evaluate them
4. Select best model
5. Perform hyperparameter tuning